# **JUMBO JUMJAI**

An Optimization Hackathon Project

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

## 1. **Data Preprocessing**

In [ ]:
!pip install pulp

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

In [ ]:
data = pd.read_csv('Wi-Fi.csv')
data.head()

,Timestamp,AP Name,Status,Total user,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost
0,3/1/2025 0:00,AP7426.ac5b.af38,"ma 662d,10h,48m",7,96.51%,37.454012,-52,NaN,Moderate,17,27.890109,4000,1000
1,3/1/2025 0:00,CMU_AGR_AGRI_AP0182,"up 14d,19h,24m",6,98.21%,95.071431,-77,NaN,Mild,15,30.697188,4000,1000
2,3/1/2025 0:00,CMU_AGR_AGRI_AP0183,"up 70d,13h,4m",3,98.85%,73.199394,-71,High,Severe,7,33.415107,4000,1000
3,3/1/2025 0:00,CMU_AGR_AGRI_AP0184,"up 4d,8h,9m",8,98.84%,59.865848,-56,NaN,Moderate,20,78.562128,4000,1000
4,3/1/2025 0:00,CMU_AGR_AGRI_AP0185,"up 48d,7h,59m",2,96.78%,15.601864,-50,NaN,Mild,5,13.745678,4000,1000


In [ ]:
# Split "AP Name" into components
dfsplit = data["AP Name"].str.split("_", expand=True)
dfsplit.columns = ["AP1", "AP2", "AP3", "APID"][:dfsplit.shape[1]]

# Combine "AP1", "AP2", and "AP3" into a single column
dfsplit["AP Name"] = dfsplit[["AP1", "AP2", "AP3"]].fillna('').agg('_'.join, axis=1).str.strip('_')

# Drop "AP1", "AP2", and "AP3"
dfsplit = dfsplit.drop(columns=["AP1", "AP2", "AP3"])

# Combine with the remaining columns
data = pd.concat([
    data[["Timestamp", "Status", "Total user", "Availability",
        "Bandwidth Utilization", "Signal Strength (RSSI)",
        "Packet Loss & Latency", "Interference Level", "Device Count",
        "Session Duration", "Installation Cost", "Maintenance Cost"]],
    dfsplit
], axis=1)

# Reorder columns to move 'APID' and 'AP_Combined' to the front
data = data[["APID", "AP Name"] + [col for col in data.columns if col not in ["APID", "AP Name"]]]

In [ ]:
# Extract the last word from "AP Name" to "APID"
data['Last_Word_AP_Name'] = data['AP Name'].str.split('_').str[-1]
data['APID'] = data['Last_Word_AP_Name'] + '_' + data['APID']
data = data.drop(columns=['Last_Word_AP_Name'])

# Rename APID where None to AP0100
data.loc[data["APID"].isnull(), "APID"] = "AP0000"

# Convert Maintenance Cost to 0 where Status is up and running
data.loc[data['Status'].str.contains('up', case=False), 'Maintenance Cost'] = 0

# Split Timestamp to Date and Time
data[['Date', 'Time']] = data['Timestamp'].str.split(' ', expand=True)
data = data.drop('Timestamp', axis=1)
data['Date'] = pd.to_datetime(data['Date'])
data['Time'] = pd.to_datetime(data['Time'], errors='coerce').dt.time

# Convert 'Availability' to numeric, handling errors
data['Availability'] = pd.to_numeric(data['Availability'].str.replace('%', ''), errors='coerce')

# Update 'Total user' and 'Device Count' based on 'Status'
data.loc[data['Status'].str.contains('ma|down', case=False, na=False), ['Total user', 'Device Count']] = 0
data.head()

<ipython-input-34-c4ce366a547c>:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Time'] = pd.to_datetime(data['Time'], errors='coerce').dt.time


,APID,AP Name,Status,Total user,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,Date,Time
0,AP0000,AP7426.ac5b.af38,"ma 662d,10h,48m",0,96.51,37.454012,-52,NaN,Moderate,0,27.890109,4000,1000,2025-03-01,00:00:00
1,AGRI_AP0182,CMU_AGR_AGRI,"up 14d,19h,24m",6,98.21,95.071431,-77,NaN,Mild,15,30.697188,4000,0,2025-03-01,00:00:00
2,AGRI_AP0183,CMU_AGR_AGRI,"up 70d,13h,4m",3,98.85,73.199394,-71,High,Severe,7,33.415107,4000,0,2025-03-01,00:00:00
3,AGRI_AP0184,CMU_AGR_AGRI,"up 4d,8h,9m",8,98.84,59.865848,-56,NaN,Moderate,20,78.562128,4000,0,2025-03-01,00:00:00
4,AGRI_AP0185,CMU_AGR_AGRI,"up 48d,7h,59m",2,96.78,15.601864,-50,NaN,Mild,5,13.745678,4000,0,2025-03-01,00:00:00


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172368 entries, 0 to 172367
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   APID                    172368 non-null  object        
 1   AP Name                 172368 non-null  object        
 2   Status                  172368 non-null  object        
 3   Total user              172368 non-null  int64         
 4   Availability            172368 non-null  float64       
 5   Bandwidth Utilization   172368 non-null  float64       
 6   Signal Strength (RSSI)  172368 non-null  int64         
 7   Packet Loss & Latency   129720 non-null  object        
 8   Interference Level      128736 non-null  object        
 9   Device Count            172368 non-null  int64         
 10  Session Duration        172368 non-null  float64       
 11  Installation Cost       172368 non-null  int64         
 12  Maintenance Cost        172368

**Define uninstallation costs based on Installation Cost**

In [ ]:
uninstallation_costs = {
    3000: 450,  # For Installation Cost 3000
    4000: 600,  # For Installation Cost 4000
    5000: 750   # For Installation Cost 5000
}

# Add the 'Uninstallation Cost' column
data['Uninstallation Cost'] = data['Installation Cost'].map(uninstallation_costs)
data.head()

,APID,AP Name,Status,Total user,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost,Date,Time,Uninstallation Cost
0,AP0000,AP7426.ac5b.af38,"ma 662d,10h,48m",0,96.51,37.454012,-52,NaN,Moderate,0,27.890109,4000,1000,2025-03-01,00:00:00,600
1,AGRI_AP0182,CMU_AGR_AGRI,"up 14d,19h,24m",6,98.21,95.071431,-77,NaN,Mild,15,30.697188,4000,0,2025-03-01,00:00:00,600
2,AGRI_AP0183,CMU_AGR_AGRI,"up 70d,13h,4m",3,98.85,73.199394,-71,High,Severe,7,33.415107,4000,0,2025-03-01,00:00:00,600
3,AGRI_AP0184,CMU_AGR_AGRI,"up 4d,8h,9m",8,98.84,59.865848,-56,NaN,Moderate,20,78.562128,4000,0,2025-03-01,00:00:00,600
4,AGRI_AP0185,CMU_AGR_AGRI,"up 48d,7h,59m",2,96.78,15.601864,-50,NaN,Mild,5,13.745678,4000,0,2025-03-01,00:00:00,600


# 2. 24 Hours Average Data
Convert 172368 rows (24 hrs) -> 7182 rows (24 hrs avg)

In [ ]:
# Convert Object by using Mode
def most_frequent(series):
    return series.mode()[0] if not series.mode().empty else None

data = data.groupby(["APID", "Date"]).agg({
            "AP Name": "first",
            "Status": "first",
            "Total user": "mean",
            "Availability": "mean",
            "Bandwidth Utilization": "mean",
            "Signal Strength (RSSI)": "mean",
            "Packet Loss & Latency": most_frequent,
            "Interference Level": most_frequent,
            "Device Count": "mean",
            "Session Duration": "mean",
            "Installation Cost": "first",
            "Maintenance Cost": "first"
        }).reset_index()

data.head()

,APID,Date,AP Name,Status,Total user,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost
0,40YD_AP8359,2025-03-01,CMU_ENG_40YD,"up 84d,8h,44m",5.083333,99.698750,48.332658,-55.458333,None,Mild,9.875000,110.198275,4000,0
1,40YD_AP8360,2025-03-01,CMU_ENG_40YD,"up 11d,19h,34m",2.250000,99.172083,19.929650,-70.916667,High,None,4.083333,29.023893,4000,0
2,40YD_AP8361,2025-03-01,CMU_ENG_40YD,"up 50d,19h,34m",3.791667,99.788750,61.890893,-34.291667,High,Severe,6.250000,49.187809,4000,0
3,40YD_AP8533,2025-03-01,TOT_ENG_40YD,"up 84d,8h,54m",3.250000,97.605000,96.926939,-75.416667,High,Moderate,5.708333,101.641828,4000,0
4,40YD_AP8534,2025-03-01,TOT_ENG_40YD,"ma 147d,14h,45m",0.000000,99.772083,83.879881,-34.833333,None,Severe,0.000000,27.252927,4000,1000


Rounding Total Average User to Integer

In [ ]:
data = data.rename(columns={"Total user": "Total Average User"})
data["Total Average User"] = data["Total Average User"].round(0).astype(int)
data.head()

,APID,Date,AP Name,Status,Total Average User,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,Device Count,Session Duration,Installation Cost,Maintenance Cost
0,40YD_AP8359,2025-03-01,CMU_ENG_40YD,"up 84d,8h,44m",5,99.698750,48.332658,-55.458333,None,Mild,9.875000,110.198275,4000,0
1,40YD_AP8360,2025-03-01,CMU_ENG_40YD,"up 11d,19h,34m",2,99.172083,19.929650,-70.916667,High,None,4.083333,29.023893,4000,0
2,40YD_AP8361,2025-03-01,CMU_ENG_40YD,"up 50d,19h,34m",4,99.788750,61.890893,-34.291667,High,Severe,6.250000,49.187809,4000,0
3,40YD_AP8533,2025-03-01,TOT_ENG_40YD,"up 84d,8h,54m",3,97.605000,96.926939,-75.416667,High,Moderate,5.708333,101.641828,4000,0
4,40YD_AP8534,2025-03-01,TOT_ENG_40YD,"ma 147d,14h,45m",0,99.772083,83.879881,-34.833333,None,Severe,0.000000,27.252927,4000,1000


# 3. Assign Location Name to Each APID

In [ ]:
print("Total Unique APID =", data["APID"].nunique())
print("Total Unique AP Name =", data['AP Name'].nunique())

Total Unique APID = 7181
Total Unique AP Name = 166


Take a look at the Location.csv

In [ ]:
location = pd.read_csv('/content/สถานที่.csv')
location.tail()

FileNotFoundError: [Errno 2] No such file or directory: '/content/สถานที่.csv'

In [ ]:
location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   AP2_AP3    148 non-null    object
 1   Location1  148 non-null    object
 2   Location2  6 non-null      object
 3   Location3  4 non-null      object
 4   Location4  2 non-null      object
 5   Location5  1 non-null      object
 6   Location6  1 non-null      object
 7   Location7  1 non-null      object
dtypes: object(8)
memory usage: 9.4+ KB


In [ ]:
location['AP2_AP3'].unique()

array(['AGR_AGRI', 'AGR_ARTMU', 'AGR_ASSET', 'AGR_BA', 'AGR_BUS',
       'AGR_CLEANWT', 'AGR_CULT', 'AGR_EDU', 'AGR_FIN', 'AGR_GRAD',
       'AGR_LI', 'AGR_OD', 'AGR_OOP', 'AGR_PTRI', 'AGR_RIHES', 'AGR_SRI',
       'AGR_STRI', 'AGR_VET', 'COM_AKD', 'COM_ASSET', 'COM_CAMT',
       'COM_CELEBRATE', 'COM_CMUHEART', 'COM_CNOC', 'COM_COFFAK',
       'COM_COOP', 'COM_ECON', 'COM_FCMU', 'COM_HISTORY', 'COM_HM',
       'COM_ITSC', 'COM_ITSCC', 'COM_LAW', 'COM_LIBR', 'COM_MASS',
       'COM_OD', 'COM_OOP', 'COM_POL', 'COM_POST', 'COM_REG', 'COM_SAVE',
       'COM_SCB', 'COM_SCI', 'COM_SLAK', 'COM_SLDM', 'COM_SOC',
       'COM_SOCF', 'COM_SPP', 'COM_SWC', 'COM_VCMU', 'ENG_40YD',
       'ENG_AMS', 'ENG_ARC', 'ENG_ARCHER', 'ENG_CAMT', 'ENG_CONTAINER',
       'ENG_DMST', 'ENG_ENG', 'ENG_FAI', 'ENG_FLAT1', 'ENG_FLAT2',
       'ENG_FLAT3', 'ENG_FLAT4', 'ENG_FLAT7', 'ENG_FLAT8', 'ENG_FOODCEN',
       'ENG_GENT3', 'ENG_GENT4', 'ENG_GENT5', 'ENG_GENT6', 'ENG_GENT7',
       'ENG_GH', 'ENG_GYM', 'ENG_LADY

In [ ]:
# Remove "CMU_" or "TOT_" from "AP Name"
data["AP Name"] = data["AP Name"].str.replace("CMU_", "", regex=False)
data["AP Name"] = data["AP Name"].str.replace("TOT_", "", regex=False)
print("Total Unique Location =", location['AP2_AP3'].nunique())
print("Total Unique AP Name =", data['AP Name'].nunique())

Total Unique Location = 148
Total Unique AP Name = 149


In [ ]:
ap_names_data = set(data["AP Name"].unique())
ap_names_location = set(location["AP2_AP3"].unique())

# Find elements in data but not in location
diff1 = ap_names_data - ap_names_location
print("AP Names in 'data' but not in 'location':", diff1)

# Find elements in location but not in data
diff2 = ap_names_location - ap_names_data
print("AP Names in 'location' but not in 'data':", diff2)

AP Names in 'data' but not in 'location': {'AP7426.ac5b.af38'}
AP Names in 'location' but not in 'data': set()


**Merge the data and location DataFrames based on "AP Name" and "AP2_AP3"**

In [ ]:
data = pd.merge(data, location, left_on='AP Name', right_on='AP2_AP3', how='left')
data = data.drop(columns=['AP2_AP3'])
data.head()

,APID,Date,AP Name,Status,Total Average User,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,...,Session Duration,Installation Cost,Maintenance Cost,Location1,Location2,Location3,Location4,Location5,Location6,Location7
0,40YD_AP8359,2025-03-01,ENG_40YD,"up 84d,8h,44m",5,99.698750,48.332658,-55.458333,None,Mild,...,110.198275,4000,0,หอสี่สิบปี,NaN,NaN,NaN,NaN,NaN,NaN
1,40YD_AP8360,2025-03-01,ENG_40YD,"up 11d,19h,34m",2,99.172083,19.929650,-70.916667,High,None,...,29.023893,4000,0,หอสี่สิบปี,NaN,NaN,NaN,NaN,NaN,NaN
2,40YD_AP8361,2025-03-01,ENG_40YD,"up 50d,19h,34m",4,99.788750,61.890893,-34.291667,High,Severe,...,49.187809,4000,0,หอสี่สิบปี,NaN,NaN,NaN,NaN,NaN,NaN
3,40YD_AP8533,2025-03-01,ENG_40YD,"up 84d,8h,54m",3,97.605000,96.926939,-75.416667,High,Moderate,...,101.641828,4000,0,หอสี่สิบปี,NaN,NaN,NaN,NaN,NaN,NaN
4,40YD_AP8534,2025-03-01,ENG_40YD,"ma 147d,14h,45m",0,99.772083,83.879881,-34.833333,None,Severe,...,27.252927,4000,1000,หอสี่สิบปี,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.tail()

,APID,Date,AP Name,Status,Total Average User,Availability,Bandwidth Utilization,Signal Strength (RSSI),Packet Loss & Latency,Interference Level,...,Session Duration,Installation Cost,Maintenance Cost,Location1,Location2,Location3,Location4,Location5,Location6,Location7
7176,VET_AP7823,2025-03-01,MAE_VET,"up 1d,15h,54m",8,95.212083,34.714214,-42.666667,Low,None,...,22.247423,4000,0,แม่เหียะ ศูนย์สุขภาพสัตว์เลี้ยง,NaN,NaN,NaN,NaN,NaN,NaN
7177,VET_AP7824,2025-03-01,MAE_VET,"ma 86d,10h,44m",0,92.808333,72.860357,-73.375000,High,Moderate,...,101.999980,4000,1000,แม่เหียะ ศูนย์สุขภาพสัตว์เลี้ยง,NaN,NaN,NaN,NaN,NaN,NaN
7178,VET_AP7825,2025-03-01,MAE_VET,"up 1d,15h,54m",3,95.166667,65.684364,-32.750000,Low,Mild,...,33.601841,4000,0,แม่เหียะ ศูนย์สุขภาพสัตว์เลี้ยง,NaN,NaN,NaN,NaN,NaN,NaN
7179,WAT_AP6409,2025-03-01,ENG_WAT,"up 28d,11h,24m",4,99.258333,46.164563,-45.833333,Medium,None,...,51.556984,4000,0,วัดฝายหิน,NaN,NaN,NaN,NaN,NaN,NaN
7180,WAT_AP6667,2025-03-01,ENG_WAT,"up 4d,19h,34m",2,98.793750,7.461262,-48.666667,Medium,Mild,...,97.792896,4000,0,วัดฝายหิน,NaN,NaN,NaN,NaN,NaN,NaN


**Save the DataFrame to a CSV file**

In [ ]:
data.to_csv('processed_wifi_data.csv', index=False)

# Download the file (optional, but useful in Colab)
from google.colab import files
files.download('processed_wifi_data.csv')

# 4. **Optimization Model**
**Decision Variable:** Let
𝑥
𝑖
x
i
​
  be a binary variable (1 = install/keep AP, 0 = remove AP) for each AP.

**Objective:** MIN COST

**Constraints:**

In [ ]:
df = data

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum
df = data
# กำหนดโมเดลลดต้นทุน
model = LpProblem("WiFi_Cost_Optimization", LpMinimize)

# จำนวน AP ที่มีอยู่
num_ap = len(df)

# กำหนดประเภท AP
ap_types = {
    0: {"install_cost": 3000, "maintenance_cost": 500, "capacity": 50},
    1: {"install_cost": 4000, "maintenance_cost": 1000, "capacity": 70},
    2: {"install_cost": 5000, "maintenance_cost": 1500, "capacity": 100}
}

# กำหนดประเภท AP จาก maintenance_cost
def classify_ap_type(cost):
    for key, value in ap_types.items():
        if cost == value["maintenance_cost"]:
            return key
    return -1

df["ap_type"] = df["Maintenance Cost"].apply(classify_ap_type)
df = df[df["ap_type"] != -1]  # กรองเฉพาะ AP ที่มีข้อมูล

# กำหนด capacity ตามประเภท AP
df["capacity"] = df["ap_type"].apply(lambda x: ap_types[x]["capacity"])

# ตัวแปรตัดสินใจ (0 = ไม่ซ่อม, 1 = ซ่อม)
repair = {i: LpVariable(f"repair_{i}", cat="Binary") for i in df.index}
replace = {i: LpVariable(f"replace_{i}", cat="Binary") for i in df.index}

# ฟังก์ชันเป้าหมาย (Objective Function) → ลดค่าใช้จ่ายรวม
model += lpSum(
    df.loc[i, "Maintenance Cost"] * repair[i] + df.loc[i, "Installation Cost"] * replace[i]
    for i in df.index
)
# ข้อจำกัด (Constraints)

# 1. รองรับผู้ใช้ทั้งหมด ≥ 38,000 คน
model += lpSum(df.loc[i, "capacity"] * (repair[i] + replace[i]) for i in df.index) >= 38000

# 2. ไม่สามารถซ่อมและซื้อใหม่พร้อมกันได้
for i in df.index:
    model += repair[i] + replace[i] <= 1

# 3. จำกัดงบประมาณรวม ไม่เกิน MAX_BUDGET
MAX_BUDGET = 40000000
model += lpSum(
    df.loc[i, "Maintenance Cost"] * repair[i] + df.loc[i, "Installation Cost"] * replace[i]
    for i in df.index
) <= MAX_BUDGET

# 4. บังคับให้ APs ที่สถานะเป็น 'ma' ต้องถูกซ่อมหรือเปลี่ยน
for i in df[df["Status"] == "ma"].index:
    model += repair[i] + replace[i] == 1

# 5. ต้องรักษาคุณภาพสัญญาณเครือข่าย (RSSI ≥ -65 dBm)
model += lpSum(repair[i] * df.loc[i, "Signal Strength (RSSI)"] for i in df.index) >= -65 * lpSum(repair[i] for i in df.index)

# 6. จำกัดค่าใช้จ่ายในแต่ละ Location ไม่ให้สูงเกินไป
MAX_COST_PER_LOC = 270000
for loc in df["Location1"].unique():
    model += lpSum(
        (df.loc[i, "Maintenance Cost"] * repair[i] + df.loc[i, "Installation Cost"] * replace[i])
        for i in df[df["Location1"] == loc].index
    ) <= MAX_COST_PER_LOC

# แก้ปัญหา
model.solve()

# แสดงผลลัพธ์ในรูปแบบที่อ่านง่าย
print("Results of AP Repair and Replacement:")
for i in df.index:
    if repair[i].varValue == 1:
        print(f"Repair AP {df.loc[i, 'APID']} (Repair needed)")
    if replace[i].varValue == 1:
        print(f"Replace AP {df.loc[i, 'APID']} (Replacement needed)")

# # แสดงผลลัพธ์
# for i in df.index:
#     if repair[i].varValue == 1:
#         print(f"Repair AP {df.loc[i, 'APID']}")
#     if replace[i].varValue == 1:
#         print(f"Replace AP {df.loc[i, 'APID']}")

<ipython-input-19-9739138538ff>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["capacity"] = df["ap_type"].apply(lambda x: ap_types[x]["capacity"])


Results of AP Repair and Replacement:
Repair AP 40YD_AP8534 (Repair needed)
Repair AP 40YD_AP8535 (Repair needed)
Repair AP 40YD_AP8536 (Repair needed)
Repair AP 40YD_AP8537 (Repair needed)
Repair AP 40YD_AP8538 (Repair needed)
Repair AP 40YD_AP8539 (Repair needed)
Repair AP 40YD_AP8540 (Repair needed)
Repair AP 40YD_AP8541 (Repair needed)
Repair AP 40YD_AP8542 (Repair needed)
Repair AP 40YD_AP8543 (Repair needed)
Repair AP 40YD_AP8544 (Repair needed)
Repair AP 40YD_AP8545 (Repair needed)
Repair AP 40YD_AP8546 (Repair needed)
Repair AP 40YD_AP8547 (Repair needed)
Repair AP 40YD_AP8548 (Repair needed)
Repair AP 40YD_AP8549 (Repair needed)
Repair AP 40YD_AP8550 (Repair needed)
Repair AP 40YD_AP8551 (Repair needed)
Repair AP 40YD_AP8552 (Repair needed)
Repair AP 40YD_AP8553 (Repair needed)
Repair AP 40YD_AP8554 (Repair needed)
Repair AP 40YD_AP8555 (Repair needed)
Repair AP 40YD_AP8556 (Repair needed)
Repair AP 40YD_AP8557 (Repair needed)
Repair AP 40YD_AP8559 (Repair needed)
Repair AP 40

In [ ]:
# แสดงผลลัพธ์ที่แยกตาม Location1
print("Results of AP Repair and Replacement by Location:")
for loc in df["Location1"].unique():
    print(f"\nLocation: {loc}")
    loc_df = df[df["Location1"] == loc]  # กรองเฉพาะข้อมูลในแต่ละ Location

    for i in loc_df.index:
        if repair[i].varValue == 1:
            print(f"  Repair AP {loc_df.loc[i, 'APID']} (Repair needed)")
        if replace[i].varValue == 1:
            print(f"  Replace AP {loc_df.loc[i, 'APID']} (Replacement needed)")

Results of AP Repair and Replacement by Location:

Location: หอสี่สิบปี
  Repair AP 40YD_AP8534 (Repair needed)
  Repair AP 40YD_AP8535 (Repair needed)
  Repair AP 40YD_AP8536 (Repair needed)
  Repair AP 40YD_AP8537 (Repair needed)
  Repair AP 40YD_AP8538 (Repair needed)
  Repair AP 40YD_AP8539 (Repair needed)
  Repair AP 40YD_AP8540 (Repair needed)
  Repair AP 40YD_AP8541 (Repair needed)
  Repair AP 40YD_AP8542 (Repair needed)
  Repair AP 40YD_AP8543 (Repair needed)
  Repair AP 40YD_AP8544 (Repair needed)
  Repair AP 40YD_AP8545 (Repair needed)
  Repair AP 40YD_AP8546 (Repair needed)
  Repair AP 40YD_AP8547 (Repair needed)
  Repair AP 40YD_AP8548 (Repair needed)
  Repair AP 40YD_AP8549 (Repair needed)
  Repair AP 40YD_AP8550 (Repair needed)
  Repair AP 40YD_AP8551 (Repair needed)
  Repair AP 40YD_AP8552 (Repair needed)
  Repair AP 40YD_AP8553 (Repair needed)
  Repair AP 40YD_AP8554 (Repair needed)
  Repair AP 40YD_AP8555 (Repair needed)
  Repair AP 40YD_AP8556 (Repair needed)
  Repair

In [ ]:
# แสดงผลลัพธ์ที่แยกตาม Location1 พร้อมนับจำนวน
print("Results of AP Repair and Replacement by Location:")

for loc in df["Location1"].unique():
    print(f"\nLocation: {loc}")
    loc_df = df[df["Location1"] == loc]  # กรองเฉพาะข้อมูลในแต่ละ Location

    # ตัวแปรนับจำนวนการซ่อมและการเปลี่ยนแปลง
    repair_count = 0
    replace_count = 0

    for i in loc_df.index:
        if repair[i].varValue == 1:
            print(f"  Repair AP {loc_df.loc[i, 'APID']} (Repair needed)")
            repair_count += 1
        if replace[i].varValue == 1:
            print(f"  Replace AP {loc_df.loc[i, 'APID']} (Replacement needed)")
            replace_count += 1

    # แสดงจำนวนการซ่อมและการเปลี่ยนแปลงในแต่ละ Location
    print(f"  Total Repairs in {loc}: {repair_count}")
    print(f"  Total Replacements in {loc}: {replace_count}")

Results of AP Repair and Replacement by Location:

Location: หอสี่สิบปี
  Repair AP 40YD_AP8534 (Repair needed)
  Repair AP 40YD_AP8535 (Repair needed)
  Repair AP 40YD_AP8536 (Repair needed)
  Repair AP 40YD_AP8537 (Repair needed)
  Repair AP 40YD_AP8538 (Repair needed)
  Repair AP 40YD_AP8539 (Repair needed)
  Repair AP 40YD_AP8540 (Repair needed)
  Repair AP 40YD_AP8541 (Repair needed)
  Repair AP 40YD_AP8542 (Repair needed)
  Repair AP 40YD_AP8543 (Repair needed)
  Repair AP 40YD_AP8544 (Repair needed)
  Repair AP 40YD_AP8545 (Repair needed)
  Repair AP 40YD_AP8546 (Repair needed)
  Repair AP 40YD_AP8547 (Repair needed)
  Repair AP 40YD_AP8548 (Repair needed)
  Repair AP 40YD_AP8549 (Repair needed)
  Repair AP 40YD_AP8550 (Repair needed)
  Repair AP 40YD_AP8551 (Repair needed)
  Repair AP 40YD_AP8552 (Repair needed)
  Repair AP 40YD_AP8553 (Repair needed)
  Repair AP 40YD_AP8554 (Repair needed)
  Repair AP 40YD_AP8555 (Repair needed)
  Repair AP 40YD_AP8556 (Repair needed)
  Repair

In [ ]:
# แสดงผลลัพธ์
df["Optimal_Repair"] = [repair[i].varValue for i in df.index]
df["Optimal_Replace"] = [replace[i].varValue for i in df.index]

<ipython-input-32-3e740c6d1929>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Optimal_Repair"] = [repair[i].varValue for i in df.index]
<ipython-input-32-3e740c6d1929>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Optimal_Replace"] = [replace[i].varValue for i in df.index]


In [ ]:
# คำนวณต้นทุนรวม
total_cost = sum(
    df.loc[i, "Maintenance Cost"] * df.loc[i, "Optimal_Repair"] +
    ap_types[df.loc[i, "ap_type"]]["install_cost"] * df.loc[i, "Optimal_Replace"]
    for i in df.index
)

In [ ]:
print("Optimal Solution Found!")
print("Total Cost:", total_cost)

Optimal Solution Found!
Total Cost: 554500.0


In [ ]:
# # แสดงต้นทุนในแต่ละ Location
# print("Cost Breakdown by Location:")
# for loc in df["Location1"].unique():
#     loc_df = df[df["Location1"] == loc]  # กรองข้อมูลตาม Location1

#     # คำนวณต้นทุนสำหรับ Location นี้
#     loc_cost = sum(
#         loc_df.loc[i, "Maintenance Cost"] * loc_df.loc[i, "Optimal_Repair"] +
#         ap_types[loc_df.loc[i, "ap_type"]]["install_cost"] * loc_df.loc[i, "Optimal_Replace"]
#         for i in loc_df.index
#     )

#     print(f"\nLocation: {loc}")
#     print(f"  Total Cost for {loc}: {loc_cost}")

Cost Breakdown by Location:


In [ ]:
# สร้าง DataFrame สำหรับแสดงต้นทุนในแต่ละ Location
cost_by_location = []

for loc in df["Location1"].unique():
    loc_df = df[df["Location1"] == loc]  # กรองข้อมูลตาม Location1

    # คำนวณต้นทุนสำหรับ Location นี้
    loc_cost = sum(
        loc_df.loc[i, "Maintenance Cost"] * loc_df.loc[i, "Optimal_Repair"] +
        ap_types[loc_df.loc[i, "ap_type"]]["install_cost"] * loc_df.loc[i, "Optimal_Replace"]
        for i in loc_df.index
    )

    # เก็บข้อมูลใน list
    cost_by_location.append({"Location": loc, "Total Cost": loc_cost})

# สร้าง DataFrame จาก list ที่เก็บข้อมูลต้นทุน
cost_df = pd.DataFrame(cost_by_location)

# แสดง DataFrame ที่ได้
cost_df


,Location,Total Cost
0,หอสี่สิบปี,196000.0
1,คณะเกษตรศาสตร์,1000.0
2,แม่เหียะ คณะเกษตรศาสตร์,3000.0
3,แม่เหียะ คณะเกษตรศาสตร์ วิทยาศาสตร์และเทคโนโลย...,2000.0
4,NaN,0.0
5,สนามยิงธนู,1000.0
6,CAMT,12000.0
7,TLIC,3000.0
8,อาคารเฉลิมพระเกียรติ์ 7 รอบ,1000.0
9,AIS Playgroud,1000.0
